<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import folium
import io,sys
from shapely.geometry import Point

#popup as a website
def get_frame(url,width,height): #https://stackoverflow.com/questions/29535715/python-with-folium-how-can-i-embed-a-webpage-in-the-popup
    html = """ 
        <!doctype html>
    <html>
    <iframe id="myIFrame" width="{}" height="{}" src={}""".format(width,height,url) + """ frameborder="0"></iframe>
    <script type="text/javascript">
    var resizeIFrame = function(event) {
        var loc = document.location;
        if (event.origin != loc.protocol + '//' + loc.host) return;

        var myIFrame = document.getElementById('myIFrame');
        if (myIFrame) {
            myIFrame.style.height = event.data.h + "px";
            myIFrame.style.width  = event.data.w + "px";
        }
    };
    if (window.addEventListener) {
        window.addEventListener("message", resizeIFrame, false);
    } else if (window.attachEvent) {
        window.attachEvent("onmessage", resizeIFrame);
    }
    </script>
    </html>"""

    popup = get_frame(url,
                  width,
                  height)

    marker = folium.CircleMarker([lat,lon],
                             radius=radius,
                             color='#3186cc',
                             fill_color = '#3186cc',
                             popup=popup)

    marker.add_to(map)



    iframe = folium.element.IFrame(html=html,width=width,height=height)
    popup = folium.Popup(iframe,max_width=width)
    return popup

file='database_speleo_gucem.csv'

FILE_HEADER = ['Nom','Nom_bis','Massif','WGS84','X','Y','Alt','denivele','developpement','Equipement','Temps_approx','debutant','crue','balade']
USE_COLS = FILE_HEADER
 
df = pd.read_csv(
    file, delimiter=",", header=None,
    names=FILE_HEADER, usecols=USE_COLS,skiprows=1)

gdf = gpd.GeoDataFrame(
    df.drop(['X', 'Y'], axis=1),
    crs={'init': 'epsg:27572'},
    geometry=[Point(xy) for xy in zip(df.X, df.Y)])

caves_wgs84 = gdf.to_crs({'init': 'epsg:4326'})

test=np.array(caves_wgs84.bounds)
myMap = folium.Map(location=[45.180772, 5.716572])
for i in range(len(test[:,0])):
    text=str(df.Nom[i])+' \n Autres noms:'+str(df.Nom_bis[i])+'\n Position: \n '+str(test[i,1])+'°E \n'+ str(test[i,0])+'°N \n Altitude='+str(df.Alt[i])+'m denivele='+str(df.denivele[i])+'m \n developpement='+str(df.developpement[i])+'m'
    folium.Marker([test[i,1],test[i,0]],popup=text).add_to(myMap)


myMap.save('test.html')


from PyQt5 import QtWidgets, QtWebEngineWidgets

app = QtWidgets.QApplication(sys.argv)
    
data = io.BytesIO()
myMap.save(data, close_file=False)

w = QtWebEngineWidgets.QWebEngineView()
w.setHtml(data.getvalue().decode())
w.resize(640, 480)
w.show()

sys.exit(app.exec_()) #quitte python quand on ferme la fenetre


TypeError: must be real number, not str

In [5]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import io,sys,os
from shapely.geometry import Point
from PyQt5 import QtWidgets, QtWebEngineWidgets

# connect to the db
conn=sqlite3.connect('database.db')

c=conn.cursor()
#colname
colname=c.execute("PRAGMA table_info(cave);").fetchall()
colname = [i[1] for i in colname]
colname = colname [1:]
variablelist=globals()
for i in colname:
    temp=c.execute("SELECT "+ i +" FROM cave").fetchall()
    variablelist[i]=[i[0] for i in temp]

# ESPG
#EPSG=c.execute("SELECT EPSG FROM cave").fetchall()
#results_as_list = [i[0] for i in ESPG]
#ESPG= np.fromiter(results_as_list, dtype=np.int32)

# geodata
df=pd.DataFrame(data={'X':X,'Y':Y })
gdf = gpd.GeoDataFrame(Nom,crs={'init': 'epsg:27572'},geometry=gpd.points_from_xy(list(df.X),list(df.Y)))
caves_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
test=np.array(caves_wgs84.bounds)

# maps
myMap = folium.Map(location=[45.180772, 5.716572])
for i in range(len(Nom)):
    text=str(Nom[i]) + '\n prof : ' + str(denivele[i]) + '\n developpement : ' + str(developpement[i])
    folium.Marker([test[i,1],test[i,0]],popup=text).add_to(myMap)

myMap.save('test.html')
# app = QtWidgets.QApplication(sys.argv)
# data = io.BytesIO()
# myMap.save(data, close_file=False)
# w = QtWebEngineWidgets.QWebEngineView()
# w.setHtml(data.getvalue().decode())
# w.resize(640, 480)
# w.show()
# sys.exit(app.exec_()) #quitte python quand on ferme la fenetre


In [2]:
colname

['Nom',
 'Nom_bis',
 'Massif',
 'WGS84',
 'X',
 'Y',
 'Alt',
 'denivele',
 'developpement',
 'Equipement',
 'Temps_approx',
 'debutant',
 'crue',
 'balade',
 'Lon']